List of things that need to be implemented:
- axisymmetric variational formula and L2 error.
- $u_{\theta}=0 $ and $\frac{\partial}{\partial \theta}$=0
- boundary conditions defined well

In [144]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import pyvista

from dolfinx.fem import Constant, Function, functionspace, assemble_scalar, dirichletbc, form, locate_dofs_geometrical, locate_dofs_topological
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc
from dolfinx.io import VTXWriter
from dolfinx.mesh import create_unit_square, exterior_facet_indices, locate_entities_boundary, meshtags
from dolfinx.plot import vtk_mesh
from basix.ufl import element
from ufl import (FacetNormal, Identity, TestFunction, TrialFunction, as_matrix,as_vector, SpatialCoordinate, Measure,
                 div, dot, ds, dx, inner, lhs, nabla_grad, rhs, sym)


mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)
t = 0
T = 10
num_steps = 500
dt = T / num_steps

In [145]:
v_cg2 = element("Lagrange", mesh.topology.cell_name(), 2, shape=(mesh.geometry.dim, ))
s_cg1 = element("Lagrange", mesh.topology.cell_name(), 1)
V = functionspace(mesh, v_cg2)
Q = functionspace(mesh, s_cg1)

In [146]:
u = TrialFunction(V)
v = TestFunction(V)
p = TrialFunction(Q)
q = TestFunction(Q)

In [ ]:
tdim = mesh.topology.dim
mesh.topology.create_connectivity(tdim - 1, tdim)
boundary_facets = exterior_facet_indices(mesh.topology)

def symmetric(x):
    return np.isclose(x[0], 0)
def wall(x):
    return  np.isclose(x[0], 1)
def inflow(x):
    return np.isclose(x[1], 0)
def outflow(x):
    return np.isclose(x[1], 1)

symmetric_facets = locate_entities_boundary(mesh, tdim - 1, symmetric)
wall_facets = locate_entities_boundary(mesh, tdim - 1, wall)
inflow_facets = locate_entities_boundary(mesh, tdim - 1, inflow)
outflow_facets = locate_entities_boundary(mesh, tdim - 1, outflow)

num_facets = mesh.topology.index_map(tdim - 1).size_local
markers = np.zeros(num_facets, dtype=np.int32)
symmetric_tag = 1
wall_tag = 2
inflow_tag = 3
outflow_tag = 4
markers[symmetric_facets] = symmetric_tag
markers[wall_facets] = wall_tag
markers[inflow_facets] = inflow_tag
markers[outflow_facets] = outflow_tag
facet_marker = meshtags(mesh, tdim - 1, np.arange(num_facets, dtype=np.int32), markers)

symmetric_dofs = locate_dofs_topological(V, facet_marker.dim, facet_marker.find(symmetric_tag))

wall_dofs = locate_dofs_topological(V, facet_marker.dim, facet_marker.find(wall_tag))
u_noslip = np.array((0,) * mesh.geometry.dim, dtype=PETSc.ScalarType)
bc_noslip = dirichletbc(u_noslip, wall_dofs, V)

inflow_dofs = locate_dofs_topological(Q, facet_marker.dim, facet_marker.find(inflow_tag))
outflow_dofs = locate_dofs_topological(Q, facet_marker.dim, facet_marker.find(outflow_tag))
bc_inflow = dirichletbc(PETSc.ScalarType(4), inflow_dofs, Q)
bc_outflow = dirichletbc(PETSc.ScalarType(0), outflow_dofs, Q)

bcu = [bc_noslip]
bcp = [bc_inflow, bc_outflow]

In [148]:
u_n = Function(V)
u_n.name = "u_n"
U = 0.5 * (u_n + u)
n = FacetNormal(mesh)
f = Constant(mesh, PETSc.ScalarType((0, 0)))
k = Constant(mesh, PETSc.ScalarType(dt))
mu = Constant(mesh, PETSc.ScalarType(1))
rho = Constant(mesh, PETSc.ScalarType(1))

In [149]:
# Define strain-rate tensor

def epsilon(u):
    return sym(nabla_grad(u))

# Define stress tensor

def sigma(u, p):
    return 2 * mu * epsilon(u) - p * Identity(len(u))

# Define the variational problem for the first step
x = SpatialCoordinate(mesh)

def div_cyl(u):
    return u[0]/x[0]+u[0].dx(0)+u[1].dx(1)
def der_dir(u):
    return as_vector([u[0]*u[0].dx(0) + u[1]*u[0].dx(1) , 
                      u[0]*u[1].dx(0) + u[1]*u[1].dx(1)])

ds = Measure("ds", domain=mesh, subdomain_data=facet_marker)

p_n = Function(Q)
p_n.name = "p_n"
F1 = rho * dot((u - u_n) / k, v) *x[0]* dx
F1 += rho * dot(der_dir(u_n), v) *x[0]* dx
F1 += inner(sigma(U, p_n), epsilon(v)) *x[0]* dx
F1 += dot(p_n * n, v)* ds(1) - dot(mu * nabla_grad(U) * n, v)*ds(1) #there is a problem here I think
+ dot(p_n * n, v) *x[0]* ds(2) - dot(mu * nabla_grad(U) * n, v)*x[0]* ds(2) #this should be zero and is
+ dot(p_n * n, v) *x[0]* ds(3) - dot(mu * nabla_grad(U) * n, v)*x[0]* ds(3) #these are fine
+ dot(p_n * n, v) *x[0]* ds(4) - dot(mu * nabla_grad(U) * n, v)*x[0]* ds(4) #these are fine
F1 -= dot(f, v) *x[0]* dx
a1 = form(lhs(F1))
L1 = form(rhs(F1))

In [150]:
A1 = assemble_matrix(a1, bcs=bcu)
A1.assemble()
b1 = create_vector(L1)

In [151]:
# Define variational problem for step 2
u_ = Function(V)
a2 = form(dot(nabla_grad(p), nabla_grad(q)) *x[0]* dx)
L2 = form(dot(nabla_grad(p_n), nabla_grad(q)) *x[0]* dx - (rho / k) * div_cyl(u_) * q *x[0]* dx)
A2 = assemble_matrix(a2, bcs=bcp)
A2.assemble()
b2 = create_vector(L2)

# Define variational problem for step 3
p_ = Function(Q)
a3 = form(rho * dot(u, v) *x[0]* dx)
L3 = form(rho * dot(u_, v) *x[0]* dx - k * dot(nabla_grad(p_ - p_n), v) *x[0]* dx)
A3 = assemble_matrix(a3)
A3.assemble()
b3 = create_vector(L3)

In [152]:
# Solver for step 1
solver1 = PETSc.KSP().create(mesh.comm)
solver1.setOperators(A1)
solver1.setType(PETSc.KSP.Type.BCGS)
pc1 = solver1.getPC()
pc1.setType(PETSc.PC.Type.HYPRE)
pc1.setHYPREType("boomeramg")

# Solver for step 2
solver2 = PETSc.KSP().create(mesh.comm)
solver2.setOperators(A2)
solver2.setType(PETSc.KSP.Type.BCGS)
pc2 = solver2.getPC()
pc2.setType(PETSc.PC.Type.HYPRE)
pc2.setHYPREType("boomeramg")

# Solver for step 3
solver3 = PETSc.KSP().create(mesh.comm)
solver3.setOperators(A3)
solver3.setType(PETSc.KSP.Type.CG)
pc3 = solver3.getPC()
pc3.setType(PETSc.PC.Type.SOR)

In [153]:
from pathlib import Path
folder = Path("results")
folder.mkdir(exist_ok=True, parents=True)
vtx_u = VTXWriter(mesh.comm, folder / "poiseuille_u.bp", u_n, engine="BP4")
vtx_p = VTXWriter(mesh.comm, folder / "poiseuille_p.bp", p_n, engine="BP4")
vtx_u.write(t)
vtx_p.write(t)

In [154]:
def u_exact(x):
    values = np.zeros((2, x.shape[1]), dtype=PETSc.ScalarType)
    values[1] = 1-x[0]**2
    return values


u_ex = Function(V)
u_ex.interpolate(u_exact)

L2_error = form(dot(u_ - u_ex, u_ - u_ex) *x[0]* dx)

In [155]:
for i in range(num_steps):
    # Update current time step
    t += dt

    # Step 1: Tentative veolcity step
    with b1.localForm() as loc_1:
        loc_1.set(0)
    assemble_vector(b1, L1)
    apply_lifting(b1, [a1], [bcu])
    b1.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b1, bcu)
    solver1.solve(b1, u_.x.petsc_vec)
    u_.x.scatter_forward()

    # Step 2: Pressure corrrection step
    with b2.localForm() as loc_2:
        loc_2.set(0)
    assemble_vector(b2, L2)
    apply_lifting(b2, [a2], [bcp])
    b2.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b2, bcp)
    solver2.solve(b2, p_.x.petsc_vec)
    p_.x.scatter_forward()

    # Step 3: Velocity correction step
    with b3.localForm() as loc_3:
        loc_3.set(0)
    assemble_vector(b3, L3)
    b3.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    solver3.solve(b3, u_.x.petsc_vec)
    u_.x.scatter_forward()
    # Update variable with solution form this time step
    u_n.x.array[:] = u_.x.array[:]
    p_n.x.array[:] = p_.x.array[:]

    # Write solutions to file
    vtx_u.write(t)
    vtx_p.write(t)

    # Compute error at current time-step
    error_L2 = np.sqrt(mesh.comm.allreduce(assemble_scalar(L2_error), op=MPI.SUM))
    error_max = mesh.comm.allreduce(np.max(u_.x.petsc_vec.array - u_ex.x.petsc_vec.array), op=MPI.MAX)
    # Print error only every 20th step and at the last step
    if (i % 20 == 0) or (i == num_steps - 1):
        print(f"Time {t:.2f}, L2-error {error_L2:.2e}, Max error {error_max:.2e}")
# Close xmdf file
vtx_u.close()
vtx_p.close()
b1.destroy()
b2.destroy()
b3.destroy()
solver1.destroy()
solver2.destroy()
solver3.destroy()

Time 0.02, L2-error 3.54e-01, Max error 1.10e+00
Time 0.42, L2-error 3.07e-01, Max error 1.12e+00
Time 0.82, L2-error 3.06e-01, Max error 1.06e+00
Time 1.22, L2-error 3.07e-01, Max error 1.03e+00
Time 1.62, L2-error 3.07e-01, Max error 1.02e+00
Time 2.02, L2-error 3.07e-01, Max error 1.01e+00
Time 2.42, L2-error 3.07e-01, Max error 1.01e+00
Time 2.82, L2-error 3.08e-01, Max error 1.00e+00
Time 3.22, L2-error 3.08e-01, Max error 1.00e+00
Time 3.62, L2-error 3.08e-01, Max error 1.00e+00
Time 4.02, L2-error 3.08e-01, Max error 1.00e+00
Time 4.42, L2-error 3.08e-01, Max error 1.00e+00
Time 4.82, L2-error 3.08e-01, Max error 1.00e+00
Time 5.22, L2-error 3.08e-01, Max error 1.00e+00
Time 5.62, L2-error 3.08e-01, Max error 1.00e+00
Time 6.02, L2-error 3.08e-01, Max error 1.00e+00
Time 6.42, L2-error 3.08e-01, Max error 1.00e+00
Time 6.82, L2-error 3.08e-01, Max error 1.00e+00
Time 7.22, L2-error 3.08e-01, Max error 1.00e+00
Time 7.62, L2-error 3.08e-01, Max error 1.00e+00
Time 8.02, L2-error 

In [156]:
pyvista.start_xvfb()
topology, cell_types, geometry = vtk_mesh(V)
values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
values[:, :len(u_n)] = u_n.x.array.real.reshape((geometry.shape[0], len(u_n)))

# Create a point cloud of glyphs
function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
function_grid["u"] = values
glyphs = function_grid.glyph(orient="u", factor=0.2)

# Create a pyvista-grid for the mesh
mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(*vtk_mesh(mesh, mesh.topology.dim))

# Create plotter
plotter = pyvista.Plotter()
plotter.add_mesh(grid, style="wireframe", color="k")
plotter.add_mesh(glyphs)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    fig_as_array = plotter.screenshot("glyphs.png")

Widget(value='<iframe src="http://localhost:43247/index.html?ui=P_0x7f857241ccd0_11&reconnect=auto" class="pyv…